In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans

In [ ]:
# Cargar datasets
products = pd.read_csv('../data/raw/olist_products_dataset.csv')
categories = pd.read_csv('../data/raw/product_category_name_translation.csv') 
items = pd.read_csv('../data/raw/olist_order_items_dataset.csv')
reviews = pd.read_csv('../data/raw/olist_order_reviews_dataset.csv')

In [21]:
# Traducir nombres de categorías
products = products.merge(categories, on='product_category_name', how='left')

# Promedio de review por producto
avg_reviews = reviews.groupby('order_id')['review_score'].mean().reset_index()
items = items.merge(avg_reviews, on='order_id', how='left')

# Promedio de review por producto_id
product_reviews = items.groupby('product_id')['review_score'].mean().reset_index()
product_reviews.columns = ['product_id', 'avg_review_score']

# Unir reviews con productos
data = products.merge(product_reviews, on='product_id', how='left')

# Reemplazar NaN por valores neutros
data['avg_review_score'].fillna(3, inplace=True)
data['product_category_name_english'].fillna('Unknown', inplace=True)

data.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17660\2992114091.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['avg_review_score'].fillna(3, inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_17660\2992114091.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,avg_review_score
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery,5.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art,5.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure,5.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby,1.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares,5.0


In [22]:
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     32951 non-null  object 
 1   product_category_name          32341 non-null  object 
 2   product_name_lenght            32341 non-null  float64
 3   product_description_lenght     32341 non-null  float64
 4   product_photos_qty             32341 non-null  float64
 5   product_weight_g               32949 non-null  float64
 6   product_length_cm              32949 non-null  float64
 7   product_height_cm              32949 non-null  float64
 8   product_width_cm               32949 non-null  float64
 9   product_category_name_english  32951 non-null  object 
 10  avg_review_score               32951 non-null  float64
dtypes: float64(8), object(3)
memory usage: 2.8+ MB


product_id                         0
product_category_name            610
product_name_lenght              610
product_description_lenght       610
product_photos_qty               610
product_weight_g                   2
product_length_cm                  2
product_height_cm                  2
product_width_cm                   2
product_category_name_english      0
avg_review_score                   0
dtype: int64

In [23]:
# Rellenar nulos en columnas numéricas con la mediana
num_cols = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty',
            'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

for col in num_cols:
    data[col].fillna(data[col].median(), inplace=True)

# Verificación final
print(data.isnull().sum())


product_id                         0
product_category_name            610
product_name_lenght                0
product_description_lenght         0
product_photos_qty                 0
product_weight_g                   0
product_length_cm                  0
product_height_cm                  0
product_width_cm                   0
product_category_name_english      0
avg_review_score                   0
dtype: int64


C:\Users\ASUS\AppData\Local\Temp\ipykernel_17660\1961220000.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_17660\1961220000.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [24]:
# Selección de columnas
num_features = ['avg_review_score', 'product_weight_g', 'product_length_cm',
                'product_height_cm', 'product_width_cm', 'product_photos_qty']
cat_features = ['product_category_name_english']

# One-hot encoding
encoder = OneHotEncoder(sparse_output=False)
cat_encoded = encoder.fit_transform(data[cat_features])

# Escalado
scaler = StandardScaler()
num_scaled = scaler.fit_transform(data[num_features])

X = np.hstack([num_scaled, cat_encoded])

In [25]:
kmeans = KMeans(n_clusters=10, random_state=42)
data['cluster'] = kmeans.fit_predict(X)

# Ver los primeros productos con su cluster
print(data[['product_id', 'product_category_name_english', 'cluster']].head(10))

                         product_id product_category_name_english  cluster
0  1e9e8ef04dbcff4541ed26657ea517e5                     perfumery        2
1  3aa071139cb16b67ca9e5dea641aaa2f                           art        2
2  96bd76ec8810374ed1b65e291975717f                sports_leisure        2
3  cef67bcfe19066a932b7673e239eb23d                          baby        1
4  9dc1a7de274444849c219cff195d0b71                    housewares        4
5  41d3672d4792049fa1779bb35283ed13           musical_instruments        2
6  732bd381ad09e530fe0a5f457d81becb                    cool_stuff        3
7  2548af3e6e77a690cf3eb6368e9ab61e               furniture_decor        1
8  37cc742be07708b53a98702e77a21a02               home_appliances        1
9  8c92109888e8cdf9d66dc7e463025574                          toys        5


In [31]:

# Ejemplo con IDs reales
customer_products = data['product_id'].sample(2, random_state=42).tolist()
print("Productos del cliente:", customer_products)

# Obtener clusters de esos productos
clusters_bought = data[data['product_id'].isin(customer_products)]['cluster'].unique()
print("Clusters del cliente:", clusters_bought)

# Buscar productos del mismo cluster
recommended = data[(data['cluster'].isin(clusters_bought)) &
                   (~data['product_id'].isin(customer_products))]

print("Productos recomendados encontrados:", len(recommended))
print(recommended[['product_id', 'product_category_name_english', 'avg_review_score']].head(5))


Productos del cliente: ['f819f0c84a64f02d3a5606ca95edd272', 'b9de40e43fccb6ba53b7eadbd5c49077']
Clusters del cliente: [0 5]
Productos recomendados encontrados: 6486
                          product_id product_category_name_english  \
9   8c92109888e8cdf9d66dc7e463025574                          toys   
13  7bb6f29c2be57716194f96496660c7c2               furniture_decor   
14  eb31436580a610f202c859463d8c7415     construction_tools_safety   
16  6a2fb4dd53d2cdb88e0432f1284a004c                     perfumery   
18  a0736b92e52f6cead290e30b578413b6               furniture_decor   

    avg_review_score  
9                4.0  
13               3.0  
14               3.0  
16               3.5  
18               4.5  


In [2]:
X = np.array([
    [1, 2], [1, 4], [1, 0],
    [10, 2], [10, 4], [10, 0]
])

kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

print("Centroides:")
print(kmeans.cluster_centers_)

print("Etiquetas de cada punto:")
print(kmeans.labels_)

Centroides:
[[10.  2.]
 [ 1.  2.]]
Etiquetas de cada punto:
[1 1 1 0 0 0]
